# Building a Spam Filer with Naive Bayes

For this project, our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

To train the computer  to classify messages, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from [the The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). You can also download the dataset directly from this [link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the authors' papers.

## 1. Explore Data Set

In [ ]:
import pandas as pd

sms_spam = pd.read_csv('SMSSpamCollection', header=None, sep='\t',names=['Label', 'SMS'])
print(sms_spam.shape)
sms_spam.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
sms_spam['Label'].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

87% of messages are ham, and 13% of messages are spam.

## 2. Traning and Test Set

We are going to split the randomized dataset into a training and a test set. 

The training set should account for 80% of the dataset, and the remaining 20% of the data should be the test set.

In [ ]:
# randomly choose 80% sample from dataset for train set
# get test set by deduct train set from full set
training_set = sms_spam.sample(frac=0.8, random_state=1)
test_set  = sms_spam.drop(training_set.index)

# reset indx label for both data set
training_set = training_set.reset_index(drop=True)
test_set = test_set.reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [ ]:
training_set['Label'].value_counts(normalize=True)*100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [ ]:
test_set['Label'].value_counts(normalize=True)*100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

The percentage of spam and ham in both train set and test set are similar to the full dataset

## 3. Letter Case and puctuation

We will do initial cleaning as below:

* Remove all the punctuation from the SMS column.
* Transform every letter in every word to lower case.

In [ ]:
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [ ]:
training_set['SMS'] =training_set['SMS'].str.replace("\W"," ").str.lower()
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


 ## 4. Creating the Vocabulary

let's create a list with all of the unique words that occur in the messages of our training set.

In [ ]:
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []

for row in training_set['SMS'].iteritems():
    message = row[1]
    for str in message:
        vocabulary.append(str)
    
vocabulary=sorted(list(set(vocabulary)),reverse=True)

In [ ]:
len(vocabulary)

7783

##  5. Final Traning Set

On the previous screen, we managed to create the vocabulary for our messages in the training set. Now we're going to use the vocabulary to make the data transformation we need:


<img src="https://dq-content.s3.amazonaws.com/433/cpgp_dataset_3.png"  width="600">



<br>
We're going to use the vocabulary to  build a dictionary that we'll then convert to the DataFrame we need.

In [ ]:
# build a  unique word counts dictionary using Python dictionary comprehension, 
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts[word][index] += 1

training_set_clean = pd.concat([training_set, word_counts], axis=1)

In [ ]:
training_set_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## 6. Calculating Constants First

Now that we're done with data cleaning and have a training set to work with, we can begin creating the spam filter. As a start, let's first calculate:

* P(Spam) and P(Ham)
* NSpam, NHam, NVocabulary

We'll also use Laplace smoothing and set α = 1

In [ ]:
# Calculate P(Spam) and P(Ham). 
spam_msg = training_set_clean[training_set_clean['Label'] == 'spam']
ham_msg = training_set_clean[training_set_clean['Label'] =='ham']

p_spam = len(spam_msg)/len(training_set_clean)
print(p_spam)
p_ham = len(ham_msg)/len(training_set_clean)
print(p_ham)


0.13458950201884254
0.8654104979811574


In [ ]:
# Calculate NSpam, NHam, NVocabulary. 
n_spam = spam_msg['SMS'].apply(len).sum()
print("n_spam =",n_spam)

n_ham = ham_msg['SMS'].apply(len).sum()
print("n_ham =",n_ham)

n_vocabulary = len(vocabulary)
print("n_vocabulary =",n_vocabulary)

# Laplace smoothing
alpha = 1

n_spam = 15190
n_ham = 57237
n_vocabulary = 7783


## 7. Calculating Parameters

Let's now calculate all the parameters using the equations below:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}


In [ ]:
# Initialize two parameter dictionaries
parameter_spam= {key:0 for key in vocabulary}
parameter_ham= {key:0 for key in vocabulary}

# calculate parameter
for word in vocabulary:
    n_word_given_spam = spam_msg[word].sum()
    p_word_given_spam =  (n_word_given_spam + alpha)/ (n_spam + alpha*n_vocabulary)
    parameter_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_msg[word].sum()
    p_word_given_ham =  (n_word_given_ham + alpha)/ (n_ham + alpha*n_vocabulary)
    parameter_ham[word] = p_word_given_ham

## 8. Classifiying A New Message

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

* Takes in as input a new message (w1, w2, ..., wn)
* Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
* Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
  *  If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
  *  If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
  *  If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

To write the code we need for calculating p_spam_given_message and p_ham_given_message, we need to use these two equations:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

In [ ]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
     
    for word in message:
        if word in parameter_spam:
            p_spam_given_message *= parameter_spam[word]
        if word in parameter_ham:
            p_ham_given_message *= parameter_ham[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [ ]:
classify('WINNER!! This is the secret code to unlock the money: C3421.' )

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [ ]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## 9. Measuring the Spam Filter's Accuracy

We have created a spam filter, and we classified two new messages. We'll now try to determine how well the spam filter does on our test set of 1,114 messages.

We'll change the classify() function that we wrote previously to return the labels instead of printing them. 

we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use <B>accuracy</B> as a metric:


\begin{equation}
\text{Accuracy} = \frac{\text{number of correctly classified messages}}{\text{total number of classified messages}}
\end{equation}

In [ ]:
import re

def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
     
    for word in message:
        if word in parameter_spam:
            p_spam_given_message *= parameter_spam[word]
        if word in parameter_ham:
            p_ham_given_message *= parameter_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [ ]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set) 

In [ ]:
correct = 0
total = len(test_set)

for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct +=1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


The accuracy is 98.7%, which is very good. Our spam filter looked at 1,114 messages in the test set and classfied 1100 correct.

## Conclusion

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set, which is an excellent result. We initially aimed for an accuracy of over 80%, but we managed to do way better than that